### <center>  Chest X-rays Detection

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

Hello World!


Path folder:

cd ../../Vie_adulte/Travail/Projets/Machine_Learning/Projet1/
